In [5]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4-1106-preview",
    logprobs=True, top_logprobs=5,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who is the president of the United States?"}
  ]
)

print(completion.choices[0].message.content)

As of my last update in April 2023, the President of the United States is Joseph R. Biden Jr. He took office on January 20, 2021. If there have been any changes in the presidency after this date, I would not be aware of them. It's always a good idea to check the latest news for the most current information.


In [6]:
from IPython.core.display import HTML
# put styles into jupyter context
HTML(""" <style>.tt {
        position: relative;
        border-radius: 0.3em;
    }

    .tt .ttt {
        visibility: hidden;
        background-color: #DDD;
        color: #111;
        border: 1.5px solid gray;
        border-radius: 6px;
        padding: 5px;
        position: absolute;
        z-index: 1;
        top: 110%;
        left: 50%;
        margin-left: -60px;
    }

    .tt:hover {
        border: 1.5px solid red;
    }

    .tt:hover .ttt {
        visibility: visible;
    }

    .sample {
        margin: 6em;
        font-family: -apple-system, BlinkMacSystemFont, sans-serif;
        font-size: 1.1em;
        margin-bottom: 15em;
    }</style>""")

In [7]:
import matplotlib
from matplotlib.colors import ListedColormap
import numpy as np
import seaborn as sns
import math

# my pastel color palette in green-yellow-red space
colors = [
 '#8cd8b4',
 '#8cd8a0',
 '#b0d88c',
 '#cbd88c',
 '#d8c88c',
 '#d8ae8c',
 '#d8968c',
 '#d88c9d']

def experimental_color(prob, mse, options):
    
    if len(options)==0 or prob > -0.005:
        return colors[0]
    
    
    # distance
    highest = max(options)
    
    delta = abs(highest) - abs(prob)
    

    if delta > 15:
        return colors[0]
    if delta > 8:
        return colors[1]
    if delta > 4:
        return colors[2]
    if delta > 2:
        return colors[3]
    if delta > 1:
        return colors[4]
    if delta > 0.5:
        return colors[7]
    return colors[7]
        
        
from html import escape

import re
import numpy as np


def pretty_prob(prob):
    return np.round(np.exp(prob)*100,2)
    
def colored_html(output, xray):

    # Extracting tokens, logprobs and candidate tokens
    tokens_with_candidates = output.choices[0].logprobs.content


    # Generating HTML content with colored tokens and tooltips for candidate tokens
    html = "<div style='margin: 2em 5em 20em 5em;'><p><i>Hover with mouse over token</i></p>"
    for entry in tokens_with_candidates:

        mse = 0

        token = entry.token
        logprob = entry.logprob
        candidates = entry.top_logprobs or []
        
        options = []

        for cand in candidates:
            t = cand.token
            
            if t == token:
                continue
                

            stripped = re.sub(r'\W+', '', t)
            
            if not stripped:
                continue
            
            prob = cand.logprob
            mse += (logprob - prob) * (logprob - prob)
            if prob != logprob:
                options.append(prob)

        #print(token, mse)

        # Formatting the list of candidate tokens for the tooltip
        candidates_str = "<br/>".join([f"{escape(cand.token).strip()}: {pretty_prob(cand.logprob)}%" for cand in candidates])
        candidates_str =candidates_str.replace(" ", "&nbsp;")
        
        if xray:
            color = experimental_color(logprob, mse, options)
        else:
            color = "white"
        cleaned = escape(token).replace('\n','<br/>')
        
        html += f"<span class='tt' style='background-color:{color};'>{cleaned}<span class='ttt'><b>Options</b><br/>{candidates_str}</span></span>"
    html += "</div>"
    return html

HTML(colored_html(completion, True))